In [1]:
import pandas as pd

# import raw data and convert into a pivot table,
# rename duplicates,
# also create indexes for uzemi_kod and vira_kod

df = pd.read_csv("sldb2021_vira.csv")
# remove columns with only one unique value
df = df.loc[:, df.nunique() > 1]

df["vira_kod"] = df["vira_kod"].fillna(0).astype(int)
# take only vira_kod and vira_txt columns and get unique values
df_vira = df[["vira_kod", "vira_txt"]].drop_duplicates()
df_vira.to_csv("vira_index.csv", index=False)


# clean up duplicates in (uzemi_kod, uzemi_txt)
df.loc[(df["uzemi_kod"] == 3107) & (df["uzemi_txt"] == "Milevsko"), "uzemi_kod"] = (
    1113107
)
df.loc[
    (df["uzemi_kod"] == 3115) & (df["uzemi_txt"] == "Týn nad Vltavou"), "uzemi_kod"
] = 1113115
# take only uzemi_kod and uzemi_txt columns and get unique values
df_uzemi = df[["uzemi_kod", "uzemi_txt"]].drop_duplicates()
# sort by number of digits in uzemi_kod, add a nb_digits column
df_uzemi["nb_digits"] = df_uzemi["uzemi_kod"].apply(lambda x: len(str(x)))
df_uzemi = df_uzemi.sort_values(by="nb_digits", ascending=True)
df_uzemi.to_csv("uzemi_index.csv", index=False)


df = df.drop(columns=["idhod", "vira_txt", "uzemi_txt", "uzemi_cis"])

# pivot table, rows are uzemi_kod, columns are vira_kod, values are hodnota
df_pivot = df.pivot(index="uzemi_kod", columns="vira_kod", values="hodnota")
df_pivot.to_csv("docs/vira_by_uzemi.csv")